In [1]:
import pandas as pd
import numpy as np
import MySQLdb
from pandas.io import sql
from sqlalchemy import create_engine

In [2]:
with open("username.txt") as f:
    user = f.read()
with open("password.txt") as g:
    password = g.read()

In [3]:
engine = create_engine('mysql+mysqldb://' + user + ':' + password + '@127.0.0.1:3306/board_games?charset=utf8mb4')

In [27]:
user_ratings = pd.read_sql("SELECT * FROM user_ratings", engine)

In [28]:
user_ratings

,gameid,username,rating
0,30549,Torsten,10.0
1,30549,mitnachtKAUBO-I,10.0
2,30549,avlawn,10.0
3,30549,Mike Mayer,10.0
4,30549,Mease19,10.0
...,...,...,...
14064607,156089,himokl,4.0
14064608,156089,masterxavier,3.0
14064609,156089,tkzubaran,2.0
14064610,156089,Simplykoke,1.0


In [29]:
user_nrates = pd.DataFrame(user_ratings.username.value_counts())

In [30]:
user_nrates

,username
Doel,2474
leffe dubbel,2320
TomVasel,2106
JasonSaastad,2004
Hessu68,2000
...,...
Freddy_Plays,1
adamobren,1
Hiannn,1
UnicornGamer,1


In [31]:
low_raters = user_nrates.loc[user_nrates['username'] < 10].index.tolist()

In [32]:
user_ratings_subset = user_ratings[~user_ratings.username.isin(low_raters)]

In [33]:
user_ratings_subset

,gameid,username,rating
0,30549,Torsten,10.0
1,30549,mitnachtKAUBO-I,10.0
2,30549,avlawn,10.0
3,30549,Mike Mayer,10.0
4,30549,Mease19,10.0
...,...,...,...
14064607,156089,himokl,4.0
14064608,156089,masterxavier,3.0
14064609,156089,tkzubaran,2.0
14064610,156089,Simplykoke,1.0


In [34]:
user_ratings_subset.drop_duplicates(inplace=True)

C:\Users\timco\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
multis = user_ratings_subset.loc[user_ratings_subset.duplicated(subset=["gameid", "username"], keep=False)]  # rows where (game, user) is duplicate
df_no_multis = user_ratings_subset.loc[~user_ratings_subset.duplicated(subset=["gameid", "username"], keep=False)] # rows where (game, user) is NOT duplicate

In [36]:
df_ratings = df_no_multis.copy()

In [14]:
#df_ratings.to_sql(name="user_ratings_subset", con=engine, if_exists="append", index=False)

In [4]:
df_ratings = pd.read_sql("SELECT * FROM user_ratings_subset", engine)

In [5]:
df_ratings

,gameid,username,rating
0,30549,Torsten,10.0
1,30549,mitnachtKAUBO-I,10.0
2,30549,avlawn,10.0
3,30549,Mike Mayer,10.0
4,30549,Mease19,10.0
...,...,...,...
13481034,156089,himokl,4.0
13481035,156089,masterxavier,3.0
13481036,156089,tkzubaran,2.0
13481037,156089,Simplykoke,1.0


In [16]:
#df_ratings.groupby('username')['rating'].mean()

username
 beastvol         7.461538
 mycroft          7.635294
-=Yod@=-          7.407843
-Johnny-          5.585752
-Loren-           7.334177
                    ...   
zzzuzu            7.285714
zzzvone           6.920833
zzzxxxyyy         8.250000
zzzzzane          7.463571
Æleksandr Þræð    8.150000
Name: rating, Length: 186981, dtype: float64

In [18]:
#df_rating_grouping = df_ratings.groupby('username')

In [19]:
#df_rating_grouping

In [38]:
df_rating_pivot = df_ratings.pivot_table(index = 'gameid', columns = 'username', values = 'rating')

MemoryError: 

In [25]:
df_rating_pivot

gameid,1,3,5,7,9,10,11,12,13,14,...,283948,284083,284435,284760,285774,285984,286096,287954,290448,291457
username,,,,,,,,,,,,,,,,,,,,,
beastvol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mycroft,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-=Yod@=-,NaN,NaN,NaN,7.5,NaN,NaN,7.0,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Johnny-,5.0,4.0,7.0,NaN,NaN,4.0,6.0,5.0,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Loren-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzzuzu,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzzvone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzzxxxyyy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
user_means1 = df_rating_pivot.mean(axis=1)

In [27]:
#user_means0 = df_rating_pivot.mean(axis=0)

In [28]:
#user_means1

username
 beastvol         7.461538
 mycroft          7.635294
-=Yod@=-          7.407843
-Johnny-          5.585752
-Loren-           7.334177
                    ...   
zzzuzu            7.285714
zzzvone           6.920833
zzzxxxyyy         8.250000
zzzzzane          7.463571
Æleksandr Þræð    8.150000
Length: 186981, dtype: float64

In [29]:
user_means0

gameid
1         7.682495
3         7.463900
5         7.396920
7         6.494584
9         6.464057
            ...   
285984    7.758394
286096    7.471114
287954    7.853167
290448    8.472341
291457    9.173556
Length: 3000, dtype: float64

In [30]:
df_rating_pivot = df_rating_pivot.subtract(user_means1, axis = 0)

In [31]:
df_rating_pivot

gameid,1,3,5,7,9,10,11,12,13,14,...,283948,284083,284435,284760,285774,285984,286096,287954,290448,291457
username,,,,,,,,,,,,,,,,,,,,,
beastvol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.538462,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mycroft,NaN,NaN,0.364706,NaN,NaN,NaN,NaN,NaN,1.364706,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-=Yod@=-,NaN,NaN,NaN,0.092157,NaN,NaN,-0.407843,NaN,-0.407843,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Johnny-,-0.585752,-1.585752,1.414248,NaN,NaN,-1.585752,0.414248,-0.585752,1.414248,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Loren-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzzuzu,NaN,NaN,NaN,NaN,NaN,NaN,1.714286,NaN,1.714286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzzvone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.079167,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzzxxxyyy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_rating_pivot.fillna(0)

gameid,1,3,5,7,9,10,11,12,13,14,...,283948,284083,284435,284760,285774,285984,286096,287954,290448,291457
username,,,,,,,,,,,,,,,,,,,,,
beastvol,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.538462,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mycroft,0.000000,0.000000,0.364706,0.000000,0.0,0.000000,0.000000,0.000000,1.364706,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-=Yod@=-,0.000000,0.000000,0.000000,0.092157,0.0,0.000000,-0.407843,0.000000,-0.407843,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Johnny-,-0.585752,-1.585752,1.414248,0.000000,0.0,-1.585752,0.414248,-0.585752,1.414248,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Loren-,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzzuzu,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.714286,0.000000,1.714286,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzzvone,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.079167,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzzxxxyyy,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
from scipy.sparse import csr_matrix


In [3]:
df_sparse_matrix = csr_matrix(df_rating_pivot.values)

NameError: name 'df_rating_pivot' is not defined

In [6]:
df_rating_pivot2 = df_ratings.groupby(['gameid', 'username'])['rating'].max().unstack()

In [7]:
df_rating_pivot2

username,beastvol,mycroft,-=Yod@=-,-Johnny-,-Loren-,-LucaS-,-mIDE-,-mik-,-pj-,-snarf-,...,zzz2525,zzzabiss,zzzeagle,zzzk1,zzzkardel,zzzuzu,zzzvone,zzzxxxyyy,zzzzzane,Æleksandr Þræð
gameid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,4.0,NaN,NaN,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,8.0,NaN,7.0,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
user_means1b = df_rating_pivot2.mean(axis=1)

In [9]:
df_rating_pivot2 = df_rating_pivot2.subtract(user_means1b, axis = 0)

In [10]:
df_rating_pivot2

username,beastvol,mycroft,-=Yod@=-,-Johnny-,-Loren-,-LucaS-,-mIDE-,-mik-,-pj-,-snarf-,...,zzz2525,zzzabiss,zzzeagle,zzzk1,zzzkardel,zzzuzu,zzzvone,zzzxxxyyy,zzzzzane,Æleksandr Þræð
gameid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,-2.682495,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,-3.463900,NaN,NaN,0.5361,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,0.60308,NaN,-0.396920,NaN,NaN,NaN,-0.39692,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,1.005416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.528886,-0.471114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.853167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_rating_pivot2 = df_rating_pivot2.fillna(0)

In [12]:
df_rating_pivot2

username,beastvol,mycroft,-=Yod@=-,-Johnny-,-Loren-,-LucaS-,-mIDE-,-mik-,-pj-,-snarf-,...,zzz2525,zzzabiss,zzzeagle,zzzk1,zzzkardel,zzzuzu,zzzvone,zzzxxxyyy,zzzzzane,Æleksandr Þræð
gameid,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.00000,0.000000,-2.682495,0.0,0.0,0.0000,0.00000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.00000,0.000000,-3.463900,0.0,0.0,0.5361,0.00000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.60308,0.000000,-0.396920,0.0,0.0,0.0000,-0.39692,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.00000,1.005416,0.000000,0.0,0.0,0.0000,0.00000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0000,0.00000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285984,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0000,0.00000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
286096,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0000,0.00000,0.0,0.0,...,2.528886,-0.471114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
287954,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0000,0.00000,0.0,0.0,...,0.000000,-0.853167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
from scipy.sparse import csr_matrix
df_sparse_matrix = csr_matrix(df_rating_pivot2.values)

In [15]:
df_games = pd.read_sql("SELECT id, name FROM games", engine)

In [16]:
df_games

,id,name
0,30549,Pandemic
1,822,Carcassonne
2,13,Catan
3,68448,7 Wonders
4,36218,Dominion
...,...,...
2995,241533,Mansions of Madness: Second Edition – Sanctum ...
2996,37235,Agricola Z-Deck
2997,8552,I Go!
2998,20542,Advanced Squad Leader: Starter Kit #3


In [17]:
game_to_idx = {
    game: i for i, game in 
    enumerate(list(df_games.set_index('id').loc[df_rating_pivot2.index].name))
}

In [18]:
game_to_idx

{'Die Macher': 0,
 'Samurai': 1,
 'Acquire': 2,
 'Cathedral': 3,
 'El Caballero': 4,
 'Elfenland': 5,
 'Bohnanza': 6,
 'Ra': 7,
 'Catan': 8,
 'Basari': 9,
 'Cosmic Encounter': 1137,
 'RoboRally': 11,
 'Wacky Wacky West': 12,
 'Magic Realm': 13,
 'Age of Renaissance': 14,
 'Supremacy: The Game of the Superpowers': 15,
 'Illuminati (Second Edition)': 16,
 'Dark Tower': 17,
 "Can't Stop": 18,
 'Tigris & Euphrates': 19,
 'David & Goliath': 20,
 'Perudo': 21,
 'Medici': 22,
 'Chinatown': 23,
 'Mamma Mia!': 24,
 'Lost Cities': 25,
 'Ricochet Robots': 26,
 'Tikal': 27,
 'Kings & Things': 28,
 'Giganten': 29,
 'Vinci': 30,
 'Löwenherz': 31,
 'Big City': 32,
 'Civilization': 33,
 'Verräter': 34,
 'Show Manager': 35,
 'Apples to Apples': 36,
 'Falling': 37,
 'Torres': 38,
 'Paths of Glory': 39,
 'El Grande': 40,
 'Union Pacific': 41,
 'Igel Ärgern': 42,
 'Conquest of the Empire': 744,
 'Axis & Allies': 629,
 'Fortress America': 45,
 'Detroit-Cleveland Grand Prix': 46,
 'Titan': 47,
 'Colossal Ar

In [19]:
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz

In [20]:
%env JOBLIB_TEMP_FOLDER=/tmp
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-2)
# fit
model_knn.fit(df_sparse_matrix)

env: JOBLIB_TEMP_FOLDER=/tmp


NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-2, n_neighbors=20, p=2,
                 radius=1.0)

In [21]:
def fuzzy_matching(mapper, fav_game, verbose=True):
    """
    return the closest match via fuzzy ratio. If no match found, return None
    
    Parameters
    ----------    
    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie
    
    verbose: bool, print log if True

    Return
    ------
    index of the closest match
    """
    match_tuple = []
    # get match
    for name, idx in mapper.items():
        ratio = fuzz.ratio(name.lower(), fav_game.lower())
        if ratio >= 60:
            match_tuple.append((name, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Found possible matches in our database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]



def make_recommendation(model_knn, data, mapper, fav_game, n_recommendations):
    """
    return top n similar movie recommendations based on user's input movie


    Parameters
    ----------
    model_knn: sklearn model, knn model

    data: movie-user matrix

    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie

    n_recommendations: int, top n recommendations

    Return
    ------
    list of top n similar movie recommendations
    """
    # fit
    model_knn.fit(data)
    # get input movie index
    print('You have input movie:', fav_game)
    idx = fuzzy_matching(mapper, fav_game, verbose=True)
    # inference
    print('Recommendation system start to make inference')
    print('......\n')
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    # get list of raw idx of recommendations
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_game))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance of {2}'.format(i+1, reverse_mapper[idx], dist))

In [22]:
my_favorite = 'Ticket to Ride'

make_recommendation(
    model_knn=model_knn,
    data=df_sparse_matrix,
    fav_game=my_favorite,
    mapper=game_to_idx,
    n_recommendations=10)

You have input movie: Ticket to Ride
Found possible matches in our database: ['Ticket to Ride', 'Ticket to Ride: London', 'Ticket to Ride: Europe', 'Ticket to Ride: Märklin', 'Ticket to Ride: New York', 'Ticket to Ride: USA 1910', 'Ticket to Ride: Europa 1912', 'Ticket to Ride: Switzerland', 'Ticket to Ride: Rails & Sails', 'Ticket to Ride: The Card Game', 'Ticket to Ride: Alvin & Dexter', 'T.I.M.E Stories', 'Ticket to Ride: 10th Anniversary', 'Ticket to Ride: Nordic Countries']

Recommendation system start to make inference
......

Recommendations for Ticket to Ride:
1: King of Tokyo, with distance of 0.8401937555298005
2: Stone Age, with distance of 0.8390437762405044
3: Splendor, with distance of 0.8303420429760962
4: 7 Wonders, with distance of 0.8280778150616749
5: Ticket to Ride: Nordic Countries, with distance of 0.8264507044642562
6: Pandemic, with distance of 0.7962485705647631
7: Catan, with distance of 0.7886066181736919
8: Carcassonne, with distance of 0.777196950596637
9: 

In [23]:
import scipy.sparse
scipy.sparse.save_npz('obj/sparse_matrix2.npz', df_sparse_matrix)